In [ ]:
import cv2
import numpy as np  
%matplotlib inline
from matplotlib import pyplot as plt
from mirage.mirage_helpers import *
import pickle

def showim(im):
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    plt.show()

def show_images(images, cols = 1, titles = None):
    """Display a list of images in a single figure with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    
    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, int(np.ceil(n_images/float(cols))), n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

def GetFrameFromVideo(filepath, framenumber):
    _cap = cv2.VideoCapture(filepath)
    _cap.set(cv2.CAP_PROP_POS_FRAMES, framenumber)
    ret, frame = _cap.read()
    return ret, frame

def GetCropFromVideo(filepath, framenumber):
    ret, img = GetFrameFromVideo(filepath, framenumber)
    return crophelpr(img)

number_of_squares_X = 10
number_of_squares_y = 7
nX = number_of_squares_X - 1 
nY = number_of_squares_y - 1
square_size = 0.025 # 1 inch in meters

# For calibrating front camera
# camera_angle = "front"
# filepath = "../../top_square.mp4"

# for calibrating side camera
camera_angle = "side"
filepath = "../../bottom_squares.mp4"

def crophelpr(img):
    if camera_angle == "front":
        frame = crop_image(img, 0, 720, 0, 1280, 0)
    else:
        frame = crop_image(img, 720, 720, 0, 1280, 0)
    return frame

def getcalibinfo():
    calib_result_pickle = pickle.load(open(f"{camera_angle}_camera_calib_pickle.p", "rb"))
    mtx = calib_result_pickle["mtx"]
    optimal_camera_matrix = calib_result_pickle["optimal_camera_matrix"]
    dist = calib_result_pickle["dist"]
    return mtx, optimal_camera_matrix, dist


In [ ]:
mtx, optimal_camera_matrix, dist = getcalibinfo()

def drawAxis(img, corners, imgpts):
    corner = tuple(corners[0].astype(int).ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].astype(int).ravel()), (255, 0, 0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].astype(int).ravel()), (0, 255, 0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].astype(int).ravel()), (0, 0, 255), 5)
    return img

def drawCube(img, corners, imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)
    # draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
    # draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img

print(mtx)
# print(optimal_camera_matrix)
print(dist)

In [ ]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
object_points = np.zeros((nY*nX,3), np.float32)
# object_points[:,:2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2) #7,6
object_points[:,:2] = np.mgrid[0:nY, 0:nX].T.reshape(-1, 2) # y, x chessboard


# transform axis
axis = np.float32([[3,0,0], [0,3,0], [0,0,-1]]).reshape(-1, 3)

# cube axis
# axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0], [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ])

framenum = 1 # 726
# framenum = 726 # 726

In [ ]:
ret, img = GetFrameFromVideo(filepath, framenum)
img = crophelpr(img)
img = cv2.undistort(img, mtx, dist, None, optimal_camera_matrix)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCorners(gray, (nY,nX), None)
if ret:
    print(len(corners))
    print(corners[0])
    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
    
    # find the rotation and translation vectors
    ret, rvecs, tvecs = cv2.solvePnP(object_points, corners2, mtx, dist)

    # project 3D points to image plane
    image_points, jacobian = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
    print(image_points)

    # projectedimg = drawCube(img.copy(), corners2, image_points)
    projectedimg = drawAxis(img.copy(), corners2, image_points)
    showim(projectedimg)
showim(img)
print(framenum)

In [ ]:
from tqdm.notebook import tqdm
cap = cv2.VideoCapture(filepath)
framelist = []
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
current_frame = 0
usable_checkboards = 0
for framenum in tqdm(range(int(length))):
    ret, img = cap.read()
    img = crophelpr(img)
    img = cv2.undistort(img, mtx, dist, None, optimal_camera_matrix)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nY,nX), None)
    if ret:
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        
        # find the rotation and translation vectors
        ret, rvecs, tvecs = cv2.solvePnP(object_points, corners2, mtx, dist)
    
        # project 3D points to image plane
        image_points, jacobian = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
    
        # projectedimg = drawCube(img.copy(), corners2, image_points)
        projectedimg = drawAxis(img.copy(), corners2, image_points)
        framelist.append(projectedimg)
    else:
        framelist.append(img)

In [ ]:
import subprocess
import os
def fix_mp4_encoding(unfixed_fn: str, fixed_fn: str):
    subprocess.run(["ffmpeg", "-y", "-i", unfixed_fn, fixed_fn])
    os.remove(unfixed_fn)


unfixed_fn = "UNFIXEDprojectedOUT.mp4"
output_fn = "projectedOUT.mp4"
out = cv2.VideoWriter(
    unfixed_fn,
    cv2.VideoWriter_fourcc("m", "p", "4", "v"),
    20.0,
    (framelist[-1].shape[1], framelist[-1].shape[0]),
)
for frame in framelist:
    out.write(frame)
out.release()
fix_mp4_encoding(unfixed_fn, output_fn)

In [ ]:
# show_images(framelist[:6], 2)